In [ ]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
import os

# Reading credentials directly
creds_file = open(f"/home/{os.getenv('USER')}/creds.txt", "r").read().strip().split(",")
accesskey, secretkey = creds_file[0], creds_file[1]

db_creds_file = open(f"/home/{os.getenv('USER')}/database-creds.txt", "r").read().strip().split(",")
dbusername, dbpassword = db_creds_file[0], db_creds_file[1]

# Spark configuration
conf = SparkConf()
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.3,mysql:mysql-connector-java:8.0.33')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')
conf.set('spark.hadoop.fs.s3a.access.key', accesskey)
conf.set('spark.hadoop.fs.s3a.secret.key', secretkey)
conf.set('spark.hadoop.fs.s3a.path.style.access', 'true')
conf.set('spark.hadoop.fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')
conf.set('spark.hadoop.fs.s3a.committer.magic.enabled', 'true')
conf.set('spark.hadoop.fs.s3a.committer.name', 'magic')
conf.set("spark.hadoop.fs.s3a.endpoint", "http://system54.rice.iit.edu")
conf.setMaster("spark://sm.service.consul:7077")
conf.set("spark.driver.memory", "8g")
conf.set("spark.executor.memory", "16g")
conf.set("spark.cores.max", '10')
conf.set('spark.executor.cores', '1')

# Spark session initialization
spark = SparkSession.builder.appName("rharidasu_50parquet") \
    .config('spark.driver.host', 'spark-edge.service.consul').config(conf=conf).getOrCreate()


# Reading Parquet file
df = spark.read.parquet('s3a://rharidasu/output/50.parquet')
df.printSchema()

# Writing to MySQL database
df.write \
    .format("jdbc") \
    .option("url", "jdbc:mysql://system75.rice.iit.edu:3306/rharidasu") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "dbrc50") \
    .option("user", dbusername) \
    .option("password", dbpassword) \
    .mode("overwrite") \
    .option("truncate", "true") \
    .save()

# Stop Spark session at the end
spark.stop()
